# Controlabilidade

Controlabilidade é uma propriedade física de um sistema que tem importantes consequências para o projeto de controladores.

Intuitivamente, controlabilidade é a possibilidade de realmente controlar o sistema para fazer algo que desejamos. Portanto, um sistema é não controlável se não for possível alterar os seus estados através de um sinal de entrada externo. 

Fisicamente, um sistema é controlável se for possível fazer a dinâmica dele iniciar em um vetor de estados inicial *arbitrário* $\mathbf{x}(0)$ e transitar para *qualquer* outro estado $\mathbf{x}(t_f)$ diferente em um intervalo de tempo finito $t_f$, usando um sinal de controle finito.

Matematicamente, o sistema é controlável quando a matriz de controlabilidade
$$
\begin{align}
\mathbf{\mathcal{C}} &= \left[\begin{array}{ccccc} \mathbf{B} & \mathbf{AB} & \mathbf{A^2B} & \ldots & \mathbf{A^{n-1}B}\end{array}\right]
\end{align}
$$
 tiver [posto](https://en.wikipedia.org/wiki/Rank_(linear_algebra)) $n$. Esse é o caso geral, válido inclusive se o sistema tiver mais de uma entrada. Normalmente, quanto mais entradas, mais controlável o sistema será.

Para um sistema de uma única entrada, basta que:
$$
\begin{align}
\det (\mathbf{\mathcal{C}}) &\neq 0
\end{align}
$$

## Exemplo:

$$
\begin{align}
G(s) &= \frac{s+2}{s^2+7s+12}
\end{align}
$$

$$
\begin{align}
    \dot{\mathbf{x}} &= \left[\begin{array}{rr}-7 & -12\\1 & 0\end{array}\right]\mathbf{x}+\left[\begin{array}{r}1\\0\end{array}\right]u\\
    y &= \left[\begin{array}{rr}1 & 2\end{array}\right]\mathbf{x}
\end{align}
$$

In [1]:
# Declara o sistema
import numpy as np
import control as ct
from IPython.display import display, Math
from escrever import *

 
A = np.matrix([[-7.,-12.],[1,0]])
B = np.matrix([1.,0]).T
C = np.matrix([1.,2])

sys = ct.ss(A,B,C,0)
print(sys)

<LinearIOSystem>: sys[2]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[ -7. -12.]
     [  1.   0.]]

B = [[1.]
     [0.]]

C = [[1. 2.]]

D = [[0.]]



Para um sistema de segunda ordem, a matriz de controlabilidade é dada por:
$$
    \begin{align}
        \mathcal{C} &= \left[\begin{array}{cc} \mathbf{B} & \mathbf{AB}\end{array}\right]
    \end{align}
$$

Em Python podemos calcular de duas formas. Diretamente:

In [2]:
# Atente para a função block() e o operador produto interno @
Con = np.block([B, A@B]) 
display(Math(r'\mathcal{C}=' + dispmat(Con)))

<IPython.core.display.Math object>

Ou usando a função especializada da biblioteca control, "ctrb()"

In [3]:
Con = (ct.ctrb(A,B))       # Note que só precisamos fornecer A e B
display(Math(r'\mathcal{C}=' + dispmat(Con)))

<IPython.core.display.Math object>

Para verificar controlabilidade, calcule o determinante. O sistema é controlável se ele for diferente de zero

In [4]:
np.linalg.det(Con)      # Precisamos da sub-biblioteca "linalg"

1.0

**Obs:** Você deve ser capaz de calcular e estudar a controlabilidade de sistemas *manualmente* (sem ajuda do Python), **pelo menos de sistemas até ordem 3**.

## Sistemas não controláveis

Um sistema "perde" a controlabilidade quando o sinal de controle "não consegue chegar" em um ou mais estados. Isso significa que, não importa o quanto o sinal de entrada varie, aqueles estados não controláveis nunca sofrem influência do controle. 

Observamos isso matematicamente quando há **cancelamento, ou quase cancelamento, entre pólos e zeros na função de transferência**.

Quando o estado não controlável é estável, isso não é muito problemático, pois mesmo não conseguindo controlá-lo, ele decai a zero depois de um tempo e acaba não influenciando a saída.

Controlabilidade torna-se um problema quando o estado não controlável é **instável**, pois nesse caso o sinal cresce indefinidamente e não somos capazes de impedir. 

### Exemplo:

In [5]:
# O seguinte sistema tem um quase cancelamento (verifique)
num = np.array([[10,50]])
eps = 1e-3
den = np.array([[1, (5+eps+3), (5+eps)*3]])
N = '10s+50'
D = 's^2 ' + '{:+1.5}'.format(den[0,1]) + '{:+1.5}'.format(den[0,2])
display(Math(r'\frac{' + N + '}{' + D + '}'))

# Estudo de controlabilidade:
A = np.block([-den[:,1:].T,np.array([[1],[0]])])
B = np.block(num.T)
Con = np.block([B,A@B])

display(Math('\mathcal{C}=' + dispmat(Con)))
display(Math('\det\,\mathcal{C} = ' + f'{np.linalg.det(Con)}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Para entender melhor esse resultado, considere que esse sistema seja estimulado por uma entrada degrau unitário. Podemos calcular a resposta analítica usando os resíduos.

In [6]:
# A resposta analítica é feita calculando os resíduos
import scipy
r,p,k = scipy.signal.residue(num[0,:],[*den[0,:],0])
r=r.reshape((1,r.size)).round(decimals=4)
p=p.reshape((1,p.size)).round(decimals=4)
display(Math(r'\text{Resíduos} = ' + dispmat(r)))
display(Math(r'\text{Polos} = ' + dispmat(p)))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Esse resultado basicamente diz que a saída tem uma parcela descrita como $-0.001e^{-5.001t}$

Isso ocorre porque o sistema tem um zero em $-5$ e um pólo muito próximo desse valor. Se o cancelamento fosse perfeito, o resíduo seria exatamente zero. Na prática isso dificilmente acontece. 

A consequência é que, **não importa a entrada**, esse quase cancelamento sempre vai acontecer. O resíduo será pequeno e a entrada terá uma influência limitada na parcela que contém o modo $e^{-5.001t}$.

Como o modo é estável (decai com o tempo), vemos que neste caso a parcela torna-se irrelevante. 

Agora, isso não seria o caso se o pólo fosse instável. Se ele fosse positivo, o resíduo ainda seria pequeno, o que significa que ele começaria pequeno, mas à medida que o tempo passasse, a parcela cresceria e o sistema instabilizaria. E não haveria nada que o sinal de entrada ía poder fazer. 

## **Exercício**: 

Para o sistema proposto:
1. Ache o diagrama da forma de controlador
2. Ache as matrizes de estados na forma de controlador
3. Estude a controlabilidade do sistema
$$
\begin{align}
    H(s) &=\frac{s^2-s+9}{s(s+1)(s+2)}
\end{align}
$$

**Determine também a controlabilidade do sistema**
